In [ ]:
%pip install --upgrade -r requirements.txt

Create the Pay-i and Azure OpenAI clients

In [ ]:
import os
from dotenv import load_dotenv

from openai import AzureOpenAI 
from payi import Payi

load_dotenv()

# Read the API KEY from the environment, replace the default values (the second argument) with your own keys if needed
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY", "YOUR_AZURE_OPENAI_API_KEY")

# Replace with the API version of your Azure OpenAI deployment
azure_openapi_version = os.getenv("AZURE_OPENAI_VERION", "2024-02-15-preview")

# Replace with your deployed Azure OpenAI endpoint URI
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "YOUR_AZURE_OPENAI_ENDPOINT")

# Replace with your Azure OpenAI Deployment Name, e.g. "test-4o"
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT", "YOUR_AZURE_OPENAI_DEPLOYMENT")

# Replace with your Azure OpenAI Model Name, e.g. "gpt-4o-2024-05-13"
# Pay-i requires the model name to understand the mapping from deployment to model
azure_model = os.getenv("AZURE_OPENAI_MODEL", "YOUR_AZURE_OPENAI_MODEL")

# Replace with one of the following values: "global", "datazone", or "region" depending on your azure deployment type
azure_deployment_type = None

client = AzureOpenAI(
    api_key=azure_openai_key,
    azure_endpoint=azure_endpoint,
    api_version=azure_openapi_version,
)

payi_api_key= os.getenv("PAYI_API_KEY", "YOUR_PAYI_API_KEY")

payi = Payi(
    api_key=payi_api_key,
)

Create a use case definition and limit.  Both will be used when ingesting data into Pay-i

In [ ]:
use_case_name = 'azure_openai_ingest_use_case_test'
payi.use_cases.definitions.create(name=use_case_name, description='This is a test use case')
 
result = payi.limits.create(max=10_000, limit_name='azure_openai_ingest_limit')
limit_id = result.limit.limit_id

print(f"Created limit with id {limit_id}")

Instrument the AzureOpenAI client so that the use case is associated with each inference call

In [ ]:
from payi.lib.instrument import payi_instrument

# Not passing payi as a parameter requires the env var "PAYI_API_KEY" is set, otherwise pass payi=payi
# When use_case_name is provided and use_id is not, a use instance id will be automatically created and the same id will be applied to all AzureOpenAI calls
payi_instrument(config={"use_case_name": use_case_name})

Helper method to call the AzureOpenAI client and pass additional information to Pay-i 

In [ ]:
from payi.lib.instrument import track_context

def call_azure_openai(client, limit_id, user_id):
    with track_context(route_as_resource=azure_model, resource_scope = azure_deployment_type, request_tags=['x', 'y'], user_id=user_id, limit_ids=[limit_id]):
        response = client.chat.completions.create(
            model=azure_deployment,
            messages=[{"role": "user", "content": "Say 'this is a test'"}],
            # To pass per method metadata to ingest, specify each metadata value in the call to create_headers.  
            # These additional header values will not be sent to AzureOpenAI.
            # route_as_resource and resource_scope are required to properly ingest Azure OpenAI results
        )
    
        completion = response.choices[0].message.content
        print(completion)

        # Additional calls to AzureOpenAI can be made here and will be ingested.
        # If another method is called, it's calls to AzureOpenAI will also be ingested.
        ...


Call AzureOpenAI and the results are automatically ingested by the Pay-i instrumentation

In [ ]:
call_azure_openai(client, limit_id, 'a_user_id')

See that the limit input, output, and total base cost has been updated with the ingest data

In [ ]:
response = payi.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.total.base}")

Reset a limit back to zero tracked cost

In [ ]:
print("\nState prior to reset:")
response = payi.limits.reset(limit_id=limit_id)
print(f"Limit Name: {response.limit_history.limit_name}")
print(f"Limit ID: {response.limit_history.limit_id}")
print(f"Limit Reset Timestamp: {response.limit_history.limit_reset_timestamp}")
print(f"Limit Tags: {response.limit_history.limit_tags}")
print(f"Limit Input Base Cost: {response.limit_history.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit_history.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.total.base}")